# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db, hrzn_load
from asteroid_element import load_ast_elt, load_data_impl, convert_data
import db_config

## Load all the Horizons CSV Files into JPL.HorizonsImport

In [3]:
# hrzn_load()

## Testing - Load CSV, Display as DataFrame

In [4]:
fname_txt = '../data/jpl/horizons/planets/daily/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/daily/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/daily/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/weekly/ast_0001.txt'

In [5]:
fname_csv = hrzn_txt2csv(fname_txt)

In [6]:
fname_csv

'/ssd1/tmp/mysql/jpl/010_sun.csv'

In [7]:
df = hrzn_csv2df(fname_csv)

In [8]:
pd.set_option('display.max_rows', 10)

In [9]:
df

,BodyTypeCD,BodyNumber,BodyName,IntegrationSource,JD,CalendarDateTime,delta_T,qx,qy,qz,vx,vy,vz
0,S,10,Sun,DE431mx,2440400.5,1969-06-28,39.699541,0.004503,0.000810,-0.000061,-3.517495e-07,0.000006,-1.438024e-08
1,S,10,Sun,DE431mx,2440401.5,1969-06-29,39.702104,0.004502,0.000815,-0.000061,-3.587527e-07,0.000006,-1.432102e-08
2,S,10,Sun,DE431mx,2440402.5,1969-06-30,39.704668,0.004502,0.000821,-0.000061,-3.657021e-07,0.000006,-1.426221e-08
3,S,10,Sun,DE431mx,2440403.5,1969-07-01,39.707232,0.004501,0.000827,-0.000061,-3.725998e-07,0.000006,-1.420343e-08
4,S,10,Sun,DE431mx,2440404.5,1969-07-02,39.709795,0.004501,0.000832,-0.000061,-3.794481e-07,0.000006,-1.414425e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196,S,10,Sun,DE431mx,2477596.5,2071-04-30,69.185505,-0.000708,-0.002510,-0.000002,4.158106e-06,-0.000004,-8.460845e-08
37197,S,10,Sun,DE431mx,2477597.5,2071-05-01,69.185494,-0.000704,-0.002514,-0.000002,4.164001e-06,-0.000004,-8.473888e-08
37198,S,10,Sun,DE431mx,2477598.5,2071-05-02,69.185482,-0.000700,-0.002518,-0.000002,4.169895e-06,-0.000004,-8.487000e-08
37199,S,10,Sun,DE431mx,2477599.5,2071-05-03,69.185470,-0.000696,-0.002522,-0.000002,4.175791e-06,-0.000004,-8.500189e-08


## Testing - Populate Database from CSV

In [10]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [11]:
engine = create_engine(connection_str)

In [12]:
engine.table_names()

['AsteroidElement_Numbered',
 'AsteroidElement_Unnumbered',
 'HorizonsBody',
 'HorizonsImport',
 'HorizonsTime',
 'HorizonsVectors',
 'LargeBody',
 'SmallBody']

In [13]:
# hrzn_txt2db(fname_txt)

## Load the Asteroid Orbital Elements from JPL

In [14]:
df = load_ast_elt()

  0%|          | 803/958724 [00:00<05:37, 2837.51it/s]

Making big simulation with all 958724 asteroids...


100%|██████████| 958724/958724 [02:44<00:00, 5815.76it/s]


Computing orbital elements...


 10%|▉         | 94789/958724 [00:00<00:00, 947884.12it/s]

Copying additional orbital elements to DataFrame...


100%|██████████| 958724/958724 [00:00<00:00, 1006439.30it/s]


In [16]:
df

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri,row_num
Num,,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.40,0.12,JPL 47,1.501306,1683.145658,0.003733,4.036516,4.187424,2.686118,-361.745861,0
2,2,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,4.20,0.11,JPL 37,2.742191,1687.410960,0.003724,4.681022,4.892910,2.150719,-679.537622,1
3,3,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,5.33,0.32,JPL 112,2.535135,1592.013779,0.003947,3.200114,3.545989,1.010854,-554.707814,2
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.00,0.32,JPL 35,1.846768,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940426,3
5,5,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,6.90,0.15,JPL 114,0.461775,1508.414438,0.004165,2.758769,2.909067,2.447291,-74.776891,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413590,1413590,6013 P-L,37203.0,2.280861,0.185919,0.104448,3.389951,3.411120,5.970031,17.14,0.15,JPL 5,5.823982,1258.191236,0.004994,0.204732,0.058683,0.517886,62.708374,958719
1413591,1413591,6331 P-L,59000.0,2.334910,0.282919,0.141062,6.200251,0.091040,4.539829,18.50,0.15,JPL 8,4.036234,1303.177715,0.004821,4.547935,4.044341,0.008106,361.584605,958720
1413592,1413592,6344 P-L,59000.0,2.817151,0.662446,0.081669,3.193653,4.099689,4.197056,20.40,0.15,JPL 17,3.444418,1727.084616,0.003638,5.207212,4.454574,1.010156,573.422791,958721


In [17]:
from astro_utils import anomaly_M2E_impl

In [18]:
M = df.M.values
e = df.e.values
E0 = None

In [19]:
E = anomaly_M2E_impl(M=M, e=e)

In [24]:
tau = 2.0 * np.pi

In [21]:
def anomaly_E2f_v1(E: np.ndarray, e: np.ndarray) -> np.ndarray:
    """
    Convert the eccentric anomaly E to the true anomaly f
    INPUTS:
        E: The Eccenctric anomaly
        e: The eccentricity
    OUTPUTS:
        f: The true anomaly    
    """
    # See https://en.wikipedia.org/wiki/Eccentric_anomaly
    # Use formula f = 2 arg(sqrt(1-e) cos(E/2), sqrt(1+e) sin(E/2))
    
    # Apply formula
    ecc_ratio = np.sqrt( (1.0 + e) / (1.0 - e))
    half_E = E * 0.5
    f = 2.0 * np.arctan(ecc_ratio * np.tan(half_E))
    # Shift angle returned to the interval [0, 2 pi)
    return f % tau

In [22]:
def anomaly_E2f(E: np.ndarray, e: np.ndarray) -> np.ndarray:
    """
    Convert the eccentric anomaly E to the true anomaly f
    INPUTS:
        E: The Eccenctric anomaly
        e: The eccentricity
    OUTPUTS:
        f: The true anomaly    
    """
    # See https://en.wikipedia.org/wiki/Eccentric_anomaly
    # Use formula f = 2 arg(sqrt(1-e) cos(E/2), sqrt(1+e) sin(E/2))
    
    # Apply formula
    half_E = E * 0.5
    x = np.sqrt(1.0 - e) * np.cos(half_E)
    y = np.sqrt(1.0 + e) * np.sin(half_E)    
    f = 2.0 * np.arctan2(y, x)
    # Shift angle returned to the interval [0, 2 pi)
    return f % tau

In [25]:
f_mse = anomaly_E2f(E, e)

In [26]:
f_reb = df.f.values

In [27]:
f_err = np.abs(f_mse - f_reb)

In [33]:
np.max(np.sin(f_err))

6.650360262483446e-10

In [28]:
f_err

array([5.32907052e-15, 8.88178420e-16, 0.00000000e+00, ...,
       0.00000000e+00, 0.00000000e+00, 1.22124533e-15])

In [29]:
idx = np.argmax(np.sin(f_err))

In [30]:
df.iloc[idx:idx+1]

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri,row_num
Num,,,,,,,,,,,,,,,,,,,,
1226469,1226469,2014 GH36,59000.0,2.258538,0.143281,0.056828,1.86741,1.303231,5.247814,18.9,0.15,JPL 8,4.979955,1239.765452,0.005068,2.13527,1.867411,3.170641,204.294138,771602


In [31]:
f_mse[idx]

4.979954686647644

In [34]:
f1 = anomaly_E2f_v1(E, e)

In [37]:
f = anomaly_E2f(E, e)

In [38]:
np.max(f1 - f)

8.881784197001252e-16